In [2]:
import pandas as pd
import numpy as np
from scipy.interpolate import interp1d
import duckdb
from IPython.display import display

pd.set_option('display.max_rows', None)     # Show all rows
pd.set_option('display.max_columns', None)  # Show all columns

datapath = r'/home/tuo47613/nfl-data-bowl/Sample_Data/Raw/'

In [3]:

# Play Data
playData = pd.read_csv(f'{datapath}plays.csv')

# Player Play Data
playerPlayData = pd.read_csv(rf'{datapath}player_play.csv')

# Player Data
playerData = pd.read_csv(f'{datapath}players.csv')

#Tracking Data
trackingData = pd.read_csv(f"{datapath}tracking_week_1.csv")

# df_list = []

#     # Loop through file names and read each CSV
# for week in range(1, 2):
#     file_name = f"{datapath}tracking_week_{week}.csv"
#     df = pd.read_csv(file_name)
#     df_list.append(df)

# trackingData = pd.concat(df_list, ignore_index=True)

In [38]:
main = duckdb.query(
    '''
    WITH SnapTime AS
    (
        SELECT
            gameId,
            playId,
            nflId,
            time AS snap_time,
            frameId AS snap_frame
        FROM trackingData
        WHERE
            frameType = 'SNAP'
    ),

    LineSetFrame AS 
    (
        SELECT 
            gameId, 
            playId, 
            nflId, 
            MIN(frameId) AS min_frameId
        FROM trackingData
        WHERE event = 'line_set'
        GROUP BY gameId, playId, nflId
    )

    SELECT
        player.position,
        player.displayName,
        td.nflId,
        td.gameId,
        td.playId,
        snap_frame,
        frameId,
        x,
        y,
        event
    FROM trackingData td

    LEFT JOIN playData pd
        ON td.gameId = pd.gameID
            AND td.playId = pd.playId

    LEFT JOIN playerData player
        ON td.nflId = player.nflId

    LEFT JOIN SnapTime st
        ON td.gameId = st.gameID
            AND td.playId = st.playId
            AND td.nflId = st.nflId

    LEFT JOIN LineSetFrame lsf 
        ON td.gameId = lsf.gameId 
            AND td.playId = lsf.playId 
            AND td.nflId = lsf.nflId

    WHERE 
        td.frameId >= lsf.min_frameId
        AND(frameType = 'BEFORE_SNAP' OR frameType = 'SNAP')
        AND td.displayName != 'football'
        AND pd.defensiveTeam = td.club

    ORDER BY
        td.gameId, td.playId, td.nflId
    '''
).df()

display(main.head(10))

,position,displayName,nflId,gameId,playId,snap_frame,frameId,x,y,event
0,ILB,Bobby Wagner,38577.0,2022090800,56,146,76,80.08,30.01,line_set
1,ILB,Bobby Wagner,38577.0,2022090800,56,146,77,80.09,29.99,None
2,ILB,Bobby Wagner,38577.0,2022090800,56,146,78,80.10,29.98,None
3,ILB,Bobby Wagner,38577.0,2022090800,56,146,79,80.11,29.97,None
4,ILB,Bobby Wagner,38577.0,2022090800,56,146,80,80.11,29.96,None
5,ILB,Bobby Wagner,38577.0,2022090800,56,146,81,80.11,29.95,None
6,ILB,Bobby Wagner,38577.0,2022090800,56,146,82,80.11,29.95,None
7,ILB,Bobby Wagner,38577.0,2022090800,56,146,83,80.12,29.97,None
8,ILB,Bobby Wagner,38577.0,2022090800,56,146,84,80.12,29.98,None
9,ILB,Bobby Wagner,38577.0,2022090800,56,146,85,80.12,30.00,None


In [34]:
num_points = 5  

def resample_path(group):
    """Resample (x, y) coordinates over frameId to a fixed number of points."""
    t = np.linspace(0, 1, len(group))  # Normalize frame positions
    x_interp = interp1d(t, group['x'], kind='linear', fill_value="extrapolate")
    y_interp = interp1d(t, group['y'], kind='linear', fill_value="extrapolate")

    # Generate standardized time steps
    t_new = np.linspace(0, 1, num_points)
    x_new, y_new = x_interp(t_new), y_interp(t_new)

    return np.concatenate([x_new, y_new])  # Flatten (x, y) into a single vector

# Apply resampling for each player-play-game
resampled_vectors = main.groupby(['gameId', 'playId', 'nflId']).apply(resample_path)

# Convert into DataFrame with meaningful column names
column_names = [f'path_x_{i+1}' for i in range(num_points)] + [f'path_y_{i+1}' for i in range(num_points)]
resampled_df = pd.DataFrame(resampled_vectors.tolist(), columns=column_names)

# Add back gameId, playId, and nflId
resampled_df[['gameId', 'playId', 'nflId']] = resampled_vectors.index.to_frame(index=False)

# Drop duplicates to keep one unique row per player-play-game
original_columns = ['position', 'displayName', 'nflId', 'gameId', 'playId']
main_unique = main[original_columns].drop_duplicates(subset=['gameId', 'playId', 'nflId'])

# Merge the original data with resampled paths
vector_distance_df = main_unique.merge(resampled_df, on=['gameId', 'playId', 'nflId'], how="inner")

/tmp/ipykernel_644709/1753968339.py:16: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  resampled_vectors = main.groupby(['gameId', 'playId', 'nflId']).apply(resample_path)


In [35]:
extrap_output = duckdb.query(
    '''
    SELECT
        *
    FROM vector_distance_df
    limit 100
    '''
).df()

display(extrap_output)

,position,displayName,nflId,gameId,playId,path_x_1,path_x_2,path_x_3,path_x_4,path_x_5,path_y_1,path_y_2,path_y_3,path_y_4,path_y_5
0,ILB,Bobby Wagner,38577.0,2022090800,56,80.08,80.0200,80.140,80.0850,80.08,30.01,29.8000,28.410,28.5350,28.52
1,DT,Aaron Donald,41239.0,2022090800,56,83.94,84.1750,84.270,84.1650,84.16,32.19,32.3500,32.450,32.3850,32.42
2,CB,Troy Hill,42816.0,2022090800,56,78.40,77.8600,77.490,77.2600,77.38,10.95,9.6750,9.140,8.7850,8.97
3,CB,Jalen Ramsey,43294.0,2022090800,56,79.54,79.3750,78.570,78.6200,77.96,38.80,40.4650,42.440,44.4500,43.99
4,DE,Leonard Floyd,43298.0,2022090800,56,84.17,84.1850,84.200,84.2200,84.23,36.09,36.0800,36.070,36.0800,36.10
5,DT,A'Shawn Robinson,43335.0,2022090800,56,83.57,83.9200,83.930,83.9300,83.91,26.54,26.3050,26.370,26.3500,26.37
6,SS,Taylor Rapp,47844.0,2022090800,56,79.49,80.0400,81.310,82.0900,82.84,16.34,18.2250,18.680,20.0500,21.59
7,NT,Greg Gaines,47917.0,2022090800,56,83.73,83.9600,83.960,83.9850,83.99,29.73,29.7000,29.700,29.7100,29.69
8,SS,Nick Scott,48026.0,2022090800,56,72.86,73.0800,72.570,71.3900,70.53,23.14,22.0650,21.190,21.3950,21.02
9,FS,Jordan Fuller,52607.0,2022090800,56,75.27,76.3450,74.530,72.2750,71.44,31.75,34.0350,35.670,35.8650,35.83
